In [10]:
import re
import pandas as pd
from collections import defaultdict

def parse_experiments(text: str):
    # Regex patterns
    matrix_size_re = re.compile(r"Running experiments for matrix size: (\d+ x \d+)")
    experiment_re = re.compile(r"Experiment \d+ of \d+")
    method_re = re.compile(r"Experimental results for (.+)")
    
    mask_stats_re = {
        "mask_total_nonzeros": re.compile(r"Total non-zeros in the mask: (\d+)"),
        "mask_sparsity": re.compile(r"Sparsity of the mask: ([\d\.]+)"),
    }
    
    method_stats_re = {
        "non_empty_blocks": re.compile(r"Number of non-empty blocks: (\d+)"),
        "avg_nonzeros_per_block": re.compile(r"Average number of non-zeros per block: ([\d\.]+)"),
        "block_sparsity": re.compile(r"Block sparsity.*: ([\d\.]+)%"),
        "block_dimension": re.compile(r"Block dimension (\d+x\d+): (\d+) blocks"),
    }

    results = defaultdict(lambda: defaultdict(list))
    current_matrix = None
    current_method = None
    current_exp_data = {}
    
    lines = text.splitlines()
    for line in lines:
        # Detect matrix size
        m = matrix_size_re.search(line)
        if m:
            current_matrix = m.group(1)
            continue
        
        # Detect experiment boundary -> reset
        if experiment_re.search(line):
            current_exp_data = {"matrix_size": current_matrix}
            continue

        # Mask stats
        for key, regex in mask_stats_re.items():
            m = regex.search(line)
            if m:
                current_exp_data[key] = float(m.group(1))
        
        # Method detection
        m = method_re.search(line)
        if m:
            current_method = m.group(1).strip()
            continue
        
        # Method stats
        if current_method:
            entry = {}
            for key, regex in method_stats_re.items():
                m = regex.search(line)
                if m:
                    if key == "block_dimension":
                        entry["block_dimension"] = m.group(1)
                        entry["block_count"] = int(m.group(2))
                    else:
                        entry[key] = float(m.group(1))
            if entry:
                # Store experiment+method results
                results[current_matrix][current_method].append({**current_exp_data, **entry})
    
    # Aggregate into DataFrame
    records = []
    for matrix_size, methods in results.items():
        for method, exps in methods.items():
            row = {"matrix_size": matrix_size, "method": method}
            # Average across experiments
            df_exp = pd.DataFrame(exps)
            for col in df_exp.columns:
                if col not in ["matrix_size", "method", "block_dimension"]:
                    row[col] = df_exp[col].mean()
            row["block_dimension"] = df_exp["block_dimension"].iloc[0]
            records.append(row)
    
    df = pd.DataFrame(records)
    return df

# Example usage
if __name__ == "__main__":
    with open("output.txt") as f:
        text = f.read()
    df = parse_experiments(text)
    df.drop

In [11]:
df.drop("block_dimension", axis=1)

,matrix_size,method,mask_total_nonzeros,mask_sparsity,non_empty_blocks,avg_nonzeros_per_block,block_sparsity,block_count
0,4096 x 4096,Baseline,136417.0,0.008131,19189.5,7.109020,29.28090,256.0
1,4096 x 4096,RCM,136417.0,0.008131,25763.5,5.294970,39.31200,256.0
2,4096 x 4096,Rownet_HyperGraph_Partitioning,136417.0,0.008131,19442.0,7.016615,29.66615,256.0
3,4096 x 4096,Hypergraph_Partitioning_Single_Border,136417.0,0.008131,19459.0,7.010505,29.69205,256.0
4,4096 x 4096,Hypergraph_Partitioning_Nonzero_Block_Minimiza...,136417.0,0.008131,17859.5,7.639035,27.25145,256.0
5,4096 x 4096,Hypergraph_Partitioning_Two_Constraint,136417.0,0.008131,23095.0,5.907215,35.24020,256.0
6,4096 x 4096,RCM + Hypergraph_Partitioning_Nonzero_Block_Mi...,136417.0,0.008131,22627.0,6.029300,34.52610,256.0
7,4096 x 4096,Hypergraph_Partitioning_Single_Border + Hyperg...,136417.0,0.008131,16104.0,8.505345,24.57275,256.0
8,4096 x 4096,Rownet_HyperGraph_Partitioning + Hypergraph_Pa...,136417.0,0.008131,14912.0,9.148905,22.75390,256.0
9,8192 x 8192,Baseline,294163.0,0.004383,49802.0,5.906655,18.99795,512.0


In [9]:
df.to_excel("experiment_results.xlsx", sheet_name="Results")